In [1]:
from loguru import logger
from api_models import get_models, ModelListResponse,ModelPermission,Model
response: ModelListResponse = await get_models()
model: Model = response.data[0]

%load_ext autoreload
%autoreload 2

2023-12-19 20:58:00.324 | INFO     | api_models:get_models:52 - An error occurred while making the request: All connection attempts failed


AttributeError: 'NoneType' object has no attribute 'data'

In [ ]:
model

In [ ]:
import openai
import asyncio
from openai_streaming import process_response
from typing import AsyncGenerator
import time
# Initialize API key
openai.api_key = "EMPTY"
openai.api_base = "http://127.0.0.1:28100/v1"
from api_models import get_models, ModelListResponse, Model
response: ModelListResponse = await get_models()
model: Model = response.data[0]

ml_format = """
<|im_start|>system
{system_message}<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
"""

print(model)
# Define content handler
async def content_handler(content: AsyncGenerator[str, None]):
    async for token in content:
        print(token, end="")

formatted_prompt = ml_format.format(system_message="A conversation between a user and an LLM-based AI assistant. The assistant gives helpful and honest answers.", prompt="Hello, how are you?")

async def main():
    # Request and process stream
    resp = openai.Completion.create(
        model=model['id'],
        prompt= formatted_prompt,
        stream=True,
        max_tokens=100
    )
    # await process_response(resp, content_handler)
    collected_events = []
    completion_text = ''
    # iterate through the stream of events
    for event in resp:
        event_text = event['choices'][0]['text']  # Extract the text
        completion_text += event_text  # Append the text
        print(f"\r{completion_text}", end='')  # Overwrite the current line with updated text



await main()

In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_name_or_path = "TheBloke/Dolphin-2.1-70B-AWQ"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=False)
# Load model
model = AutoAWQForCausalLM.from_quantized(model_name_or_path, fuse_layers=True,
                                          trust_remote_code=False, safetensors=True)

prompt = "Tell me about AI"
prompt_template=f'''<|im_start|>system
{system_message}<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
'''

print("*** Running model.generate:")

token_input = tokenizer(
    prompt_template,
    return_tensors='pt'
).input_ids.cuda()

# Generate output
generation_output = model.generate(
    token_input,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    max_new_tokens=512
)

# Get the tokens from the output, decode them, print them
token_output = generation_output[0]
text_output = tokenizer.decode(token_output)
print("LLM output: ", text_output)

"""
# Inference should be possible with transformers pipeline as well in future
# But currently this is not yet supported by AutoAWQ (correct as of September 25th 2023)
from transformers import pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])
"""


In [ ]:
# Install nest_asyncio if not already done
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

import openai 
import asyncio

openai.api_key = "EMPTY"
openai.api_base = "http://127.0.0.1:8100/v1"

async def main():

    print(f"USING MODEL : {model['id']}")
    description = "Create a Python script to sort a list of numbers in ascending order."
    response = openai.Completion.create(
      model=model["id"],
      prompt=f"Instruction:\n{description}",
      stream=True,
      max_tokens=20
    )
    
    try:
        for chunk in response:
            current_content = chunk["choices"][0]["delta"].get("content","")
            print(current_content)
    except Exception as e:
        print(f"An error occurred while making the request: {e}")
        if e.response is not None:
            print("Additional information:")
            print(f"Status code: {e.response.status_code}")
            print(f"Headers: {e.response.headers}")
            print(f"Content: {e.response.text}")

# Run the async function
loop = asyncio.get_event_loop()
loop.run_until_complete(main())

In [ ]:
def sort_numbers(numbers):
    return sorted(numbers)

# Use example
numbers = [67, 23, 34, 2, 89, 23, 2]
sorted_numbers = sort_numbers(numbers)
print(sorted_numbers)

In [ ]:
description

In [ ]:
# Example of an OpenAI Completion request, using the stream=True option
# https://beta.openai.com/docs/api-reference/completions/create
import time
# record the time before the request is sent
start_time = time.time()

# send a Completion request to count to 100
response = openai.Completion.create(
    model=model['id'],
    prompt='1,2,3,',
    max_tokens=2000,
    temperature=0,
    stream=True,  # this time, we set stream=True
)

# Iterate through the stream of events
for event in response:
    event_text = event['choices'][0]['text']  # Extract the text from the event
    collected_events.append(event)  # Collect the event (optional, if you need it later)
    
    # Append the streamed text to the existing output
    completion_text += event_text
    
    # Print the updated completion_text
    print(completion_text)
